## Importing libraries

In [3]:
import pandas as pd
import numpy as np
import scipy
from math import sqrt
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import os

Estimators: regressors

In [4]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn import linear_model

Estimators: classifiers

In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier

Metrics:

In [6]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import max_error
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

Cross-validation:

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
datafilenamepath = "C:\\Users\\edidd\\Documents\\Ubiqum\\Data Analytics Course\\Module_III_Wifi\\Data2\\"

## Data pre-processing

Data transformation selected is the power transformation with n=5.

Model for predicting buildings is 5NN with default distance metric.

### Reading csv

In [9]:
testset = pd.read_csv(os.path.join(datafilenamepath, "TEST_DATA.csv"), header = 0)

In [10]:
testset.head()

,WAP001,WAP002,WAP003,WAP004,WAP005,WAP006,WAP007,WAP008,WAP009,WAP010,...,WAP520,LONGITUDE,LATITUDE,FLOOR,BUILDINGID,SPACEID,RELATIVEPOSITION,USERID,PHONEID,TIMESTAMP
0,100,100,100,100,100,100,100,100,100,100,...,100,0,0,0,0,0,0,0,15,1385716325
1,100,100,100,100,100,100,100,100,100,100,...,100,0,0,0,0,0,0,0,15,1385716326
2,100,100,100,100,100,100,100,100,100,100,...,100,0,0,0,0,0,0,0,15,1385716326
3,100,100,100,100,100,100,100,100,100,100,...,100,0,0,0,0,0,0,0,15,1385716327
4,100,100,100,100,100,100,100,100,100,100,...,100,0,0,0,0,0,0,0,15,1385716327


## Pre-processing testset

In [11]:
enes = [4, 5]

In [12]:
for i in enes:
    test = testset.copy()
    test.iloc[:,:520] = test.iloc[:,:520].apply(lambda x: 10 ** (x/(10*i)), axis = 0 )
    test.iloc[:,:520] = test.iloc[:,:520].replace(10 ** (100/(10*i)), 0)
    test.iloc[:,:520] = test.iloc[:,:520].apply(lambda x: x/x.sum(), axis=1)
    test = test.fillna(0)
    datafilename = "testset" + "_n" + str(i) + ".csv"
    test.to_csv (os.path.join(datafilenamepath, datafilename), index = None, header=True)

### Importing pre-processed data

In [13]:
trainset = pd.read_csv("C:/Users/edidd/Documents/Ubiqum/Data Analytics Course/Module_III_Wifi/data/trainset_clean_trans_n5.csv", header = 0)

In [14]:
validation = pd.read_csv("C:/Users/edidd/Documents/Ubiqum/Data Analytics Course/Module_III_Wifi/data/validation_clean_trans_n5.csv", header = 0)

In [15]:
by_ref_point = trainset.groupby(["LONGITUDE","LATITUDE", "FLOOR"]).mean().reset_index()
by_ref_point_arr = pd.concat([by_ref_point.iloc[:, 3:523], by_ref_point.iloc[:,:3], by_ref_point.iloc[:, 523:]], axis = 1)
by_ref_point_arr["BUILDINGID"] = by_ref_point_arr.BUILDINGID.apply(int)

In [17]:
by_ref_point_arr.head()

,WAP001,WAP002,WAP003,WAP004,WAP005,WAP006,WAP007,WAP008,WAP009,WAP010,...,WAP520,LONGITUDE,LATITUDE,FLOOR,BUILDINGID,SPACEID,RELATIVEPOSITION,USERID,PHONEID,TIMESTAMP
0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,-7691.3384,4.864928e+06,1,0,222.0,2.0,6.000000,13.500000,1.370489e+09
1,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.014534,0.0,...,0.0,-7691.3384,4.864928e+06,2,0,224.0,2.0,6.000000,13.500000,1.370695e+09
2,0.0,0.009851,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,-7691.3384,4.864928e+06,3,0,223.0,2.0,4.703704,13.629630,1.370791e+09
3,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,-7689.5134,4.864929e+06,1,0,223.0,2.0,5.736842,13.526316,1.370518e+09
4,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.027121,0.0,...,0.0,-7689.5134,4.864929e+06,2,0,225.0,2.0,6.000000,13.500000,1.370695e+09


In [18]:
trainval = pd.concat([by_ref_point_arr, validation], axis = 0)

In [19]:
trainval.head()

,WAP001,WAP002,WAP003,WAP004,WAP005,WAP006,WAP007,WAP008,WAP009,WAP010,...,WAP520,LONGITUDE,LATITUDE,FLOOR,BUILDINGID,SPACEID,RELATIVEPOSITION,USERID,PHONEID,TIMESTAMP
0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,-7691.3384,4.864928e+06,1,0,222.0,2.0,6.000000,13.500000,1.370489e+09
1,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.014534,0.0,...,0.0,-7691.3384,4.864928e+06,2,0,224.0,2.0,6.000000,13.500000,1.370695e+09
2,0.0,0.009851,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,-7691.3384,4.864928e+06,3,0,223.0,2.0,4.703704,13.629630,1.370791e+09
3,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,-7689.5134,4.864929e+06,1,0,223.0,2.0,5.736842,13.526316,1.370518e+09
4,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.027121,0.0,...,0.0,-7689.5134,4.864929e+06,2,0,225.0,2.0,6.000000,13.500000,1.370695e+09


In [20]:
trainval.shape

(2044, 529)

In [21]:
testset = pd.read_csv(os.path.join(datafilenamepath, "testset_n5.csv"), header = 0)

In [22]:
testset.head()

,WAP001,WAP002,WAP003,WAP004,WAP005,WAP006,WAP007,WAP008,WAP009,WAP010,...,WAP520,LONGITUDE,LATITUDE,FLOOR,BUILDINGID,SPACEID,RELATIVEPOSITION,USERID,PHONEID,TIMESTAMP
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0,0,0,0,0,0,0,15,1385716325
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0,0,0,0,0,0,0,15,1385716326
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0,0,0,0,0,0,0,15,1385716326
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0,0,0,0,0,0,0,15,1385716327
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0,0,0,0,0,0,0,15,1385716327


In [23]:
testset.shape

(5179, 529)

### Predicting building

In [24]:
features = trainval.iloc[:,:520]

In [25]:
dependentlist_r = [trainval["LONGITUDE"], trainval["LATITUDE"]]

In [26]:
dependentlist_c = [trainval["BUILDINGID"], trainval["FLOOR"]]

In [27]:
test_features = testset.iloc[:,:520]

In [28]:
modelKNNC = KNeighborsClassifier(n_neighbors = 5, n_jobs= -1)
modelKNNC.fit(features, dependentlist_c[0])
prediction_building = modelKNNC.predict(test_features)

In [29]:
prediction_building = pd.Series(prediction_building)
prediction_building.describe()

count    5179.000000
mean        0.270516
std         0.515126
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         2.000000
dtype: float64

### Predicting floor

In [30]:
modelKNNC = KNeighborsClassifier(n_neighbors = 5, weights= "distance", metric = "euclidean", n_jobs= -1)
modelKNNC.fit(features, dependentlist_c[1])
prediction_floor = modelKNNC.predict(test_features)

In [31]:
prediction_floor = pd.Series(prediction_floor)
prediction_floor.describe()

count    5179.000000
mean        1.206024
std         0.667512
min         0.000000
25%         1.000000
50%         1.000000
75%         1.000000
max         4.000000
dtype: float64

### Prediction LONG

In [32]:
modelKNNR = KNeighborsRegressor(n_neighbors = 5, weights= "distance", metric = "euclidean", n_jobs= -1)
modelKNNR.fit(features, dependentlist_r[0])
prediction_long = modelKNNR.predict(test_features)

In [33]:
prediction_long = pd.Series(prediction_long)
prediction_long.describe()

count    5179.000000
mean    -7600.960254
std        79.283672
min     -7688.326261
25%     -7644.312509
50%     -7636.698533
75%     -7596.819474
max     -7309.495090
dtype: float64

### Prediction LAT

In [34]:
modelKNNR = KNeighborsRegressor(n_neighbors = 5, weights= "distance", metric = "euclidean", n_jobs= -1)
modelKNNR.fit(features, dependentlist_r[1])
prediction_lat = modelKNNR.predict(test_features)

In [35]:
prediction_lat = pd.Series(prediction_lat)
prediction_lat.describe()

count    5.179000e+03
mean     4.864935e+06
std      4.739861e+01
min      4.864757e+06
25%      4.864921e+06
50%      4.864937e+06
75%      4.864968e+06
max      4.865013e+06
dtype: float64

## Submission file

In [36]:
sub_examp.head()

NameError: name 'sub_examp' is not defined

In [37]:
submission2 = pd.concat([prediction_lat, prediction_long, prediction_floor, prediction_building], axis = 1)
submission2 = submission2.rename(columns={0 : "LATITUDE", 1 : "LONGITUDE", 2 : "FLOOR", 3 : "BUILDINGID"})

In [55]:
submission1.head()

,LATITUDE,LONGITUDE,FLOOR,BUILDINGID
0,4.864982e+06,-7595.214878,1,0
1,4.864982e+06,-7597.549310,1,0
2,4.864982e+06,-7597.542233,1,0
3,4.864982e+06,-7596.916426,1,0
4,4.864983e+06,-7597.398790,1,0


In [38]:
submission2.head()

,LATITUDE,LONGITUDE,FLOOR,BUILDINGID
0,4.864980e+06,-7600.541546,1,0
1,4.864981e+06,-7601.562436,1,0
2,4.864981e+06,-7601.555260,1,0
3,4.864981e+06,-7597.428204,1,0
4,4.864981e+06,-7600.093545,1,0


In [39]:
submission2.shape

(5179, 4)

In [40]:
submission2.to_csv(os.path.join(datafilenamepath, "sub2_EG_n5_5nn_mean.csv"), index = None, header=True)

In [41]:
temp = pd.read_csv(os.path.join(datafilenamepath, "sub2_EG_n5_5nn_mean.csv"))
temp.head()

,LATITUDE,LONGITUDE,FLOOR,BUILDINGID
0,4.864980e+06,-7600.541546,1,0
1,4.864981e+06,-7601.562436,1,0
2,4.864981e+06,-7601.555260,1,0
3,4.864981e+06,-7597.428204,1,0
4,4.864981e+06,-7600.093545,1,0


In [63]:
temp.describe()

,LATITUDE,LONGITUDE,FLOOR,BUILDINGID
count,5.179000e+03,5179.000000,5179.000000,5179.000000
mean,4.864935e+06,-7601.115152,1.223595,0.270516
std,4.713608e+01,79.479037,0.678403,0.515126
min,4.864756e+06,-7688.933719,0.000000,0.000000
25%,4.864921e+06,-7644.806392,1.000000,0.000000
50%,4.864937e+06,-7636.920886,1.000000,0.000000
75%,4.864970e+06,-7595.514587,1.000000,0.000000
max,4.865015e+06,-7309.495090,4.000000,2.000000
